In [1]:
# ----------------------------------------------------------------------------------------
# imports and initialization
# ----------------------------------------------------------------------------------------

import ee
import time
import pandas as pd
import sys
from IPython.display import display,Image
from helperFunctions import print_full
from GEEimports import dumpclean # displays all levels of a json object
from GEEimports import calcConsACCA,maskWater,calcGreenness

ee.Initialize()

# ----------------------------------------------------------------------------------------
# algorithms
# ----------------------------------------------------------------------------------------

# Map bloom detection algorithm  over collection
#  based on Ho et al. (2017) "Using Landsat to extend historical phytoplankton bloom records"
def calcAlgorithms(img):
#     # using the Landsat Automatic Cloud Cover Assessment 
#     img2 = calcConsACCA(img); 
#     yesCloud = img2.select("cloud");
    # using the FMask algorithm
    yesCloud = img.select(['fmask']).eq(2).Or(img.select(['fmask']).eq(4));
    img2 = img.addBands(yesCloud)
    yesLand = img.select(['fmask']).eq(0).Or(img.select(['fmask']).eq(3));
    
    #Apply algorithms
    img3=img2.expression("b('B4')-1.03*b('B5')").select(["B4"],['ImpNIRwithSAC']);

    #make negative values 0 
    img3=img3.where(img3.lte(0),0);
        
    #use greenness threshold:
    gness=calcGreenness(img);
    img3=img3.where(gness.eq(0),0)
    
    #Mask clouds
    img3=img3.updateMask(yesCloud.Not());
    
    return (img.addBands(img3)
            .addBands(yesCloud.rename(['fmask_cloud']))
            .addBands(yesLand.rename(['fmask_land']))
            .addBands(img.select('fmask').eq(1).rename(['fmask_water']))
           );
    #add a constant image band so that later sum reducer gets
    # total area of lakepolygon

#Function to return a null feature with bloom area, used to get historical time series
def calcBloom(img):  
    #get the bands in image that you don't want masked by water
    img_cloud = img.select('fmask_cloud');
    img_cloud = img_cloud.updateMask(img_cloud);
    img_land = img.select('fmask_land');
    img_land = img_land.updateMask(img_land);
    img_const = img.select('constant');
    img_count = img.select('count');

    #mask water in image
    img = maskWater(img)

    b = ee.Image(img.select('ImpNIRwithSAC')).reduceRegion(
            reducer= ee.Reducer.sum(),
            geometry=feat,       # NOTE THIS IS SET BEFOREHAND
            scale=res,           # NOTE THIS IS SET BEFOREHAND
            bestEffort=False,
            maxPixels=15000000);
    bloomSignal = ee.Number(b.get('ImpNIRwithSAC'))
    m = ee.Dictionary({
            'bloomSignal': bloomSignal
        })
    s = ee.Image(img.select(['ImpNIRwithSAC'])
                 .addBands(img_cloud)
                 .addBands(img_land)
                 .addBands(img_const)
                 .addBands(img_count)).reduceRegion(
            reducer=ee.Reducer.count(),
            geometry=feat,       # NOTE THIS IS SET BEFOREHAND
            scale=res,           # NOTE THIS IS SET BEFOREHAND
            bestEffort=False,
            maxPixels=15000000);
    
    # max polygon area --> get from dictionary of largest polygon areas set ahead of time
    px_max_poly = ee.Number(s.get('constant'));
    # obs polygon area --> count in constant
    px_obs_poly = ee.Number(s.get('count'));
    # obs water area --> count in algorithm
    px_obs_water = ee.Number(s.get('ImpNIRwithSAC'));
    # obs land area --> count in land
    px_obs_land = ee.Number(s.get('fmask_land'));
    # obs cloud pixels --> count in cloud
    px_obs_cloud = ee.Number(s.get('fmask_cloud'));
    # valid pixels --> obs (cloud + water + land pixels)
    featDict = m.combine(
        ee.Dictionary({
                'px_max_poly': px_max_poly,
                'px_obs_poly': px_obs_poly,
                'px_obs_water': px_obs_water,
                'px_obs_land': px_obs_land,
                'px_obs_cloud': px_obs_cloud,
                'px_res': res,
                'date': img.get('comp_mid'),
                'comp_num_images': img.get('comp_num_images'),
            })
                        );
    return ee.Feature(None, featDict);

In [2]:
# meta parameters
res = 30
validation_regions = True
exp_to_csv = False

In [3]:
# ----------------------------------------------------------------------------------------
# set up collections for algorithm application 
# ----------------------------------------------------------------------------------------

# Import GLWD polygons (Lehner and Doll, 2004) that have been matched to 
#  GLTC lake information (Sharma 2015) and checked for validity:

# Fusion Table with manually-corrected polygons: "StudyLakePolygons_ManuallyCorrected"
# https://fusiontables.google.com/DataSource?docid=1pB9qGGnYixNVifTBIyEvgPpsMSqxPRT7dU-uwwrv
fc_shape = ee.FeatureCollection( 
  "ft:1pB9qGGnYixNVifTBIyEvgPpsMSqxPRT7dU-uwwrv", 
  "geometry");
# Fusion Table with lake auxiliary info: "GLWD_join_GLTC"
# https://fusiontables.google.com/DataSource?docid=1EDVcrSyzzmInP7lbAb2FbJvoU9-UWMcnuYbnQuLt
fc_meta = ee.FeatureCollection(
  "ft:1EDVcrSyzzmInP7lbAb2FbJvoU9-UWMcnuYbnQuLt", 
  "geometry",)
        
# Prints all lake names in fc_shape
pointList = fc_shape.toList(350).map(
    lambda (f): ee.Feature(f).get("lakename")
);
pointList_local = sorted(pointList.getInfo())
print('Lake names in fc_shape (%s):' %len(pointList_local) )
p=pd.DataFrame(pointList_local,columns=['lakename'])
print_full(p)

Lake names in fc_shape (95):
                          lakename
0                            Abaya
1                            Ai-pi
2                           Alakol
3                           Albert
4                      Alexandrina
5                         Aral-Sea
6                        Argentina
7                           Argyle
8                          Ayakkum
9                      Bahral-Milh
10                    Baikal-North
11                         Balaton
12                        Balkhash
13                     Barun-Torey
14                             Bay
15                          Beloye
16                        Beysehir
17                 BoengTonleChhma
18                          Bosten
19                       Bratskoye
20                            Buyr
21                     CaboraBassa
22  Cha-jihNan-mu-tso--Zhari-Namco
23                            Chao
24                         Chapala
25                  Chardarinskoye
26                        

In [ ]:
n = len(pointList_local);
# original code loops through all 95 lakes above
# for i in range(0, n): 
# This shows what the output looks like for Clear Lake
for i in [30]:
    try:
        tic = time.time();
        lakename = pointList_local[i]

        # Get the lake polygon
        print('Lake name: %s' % lakename)
        lakerow_shape = fc_shape.filterMetadata('lakename', 'equals', lakename).first();

        # Get the lake metadata
        lakerow = fc_meta.filterMetadata('Lake_nam_1', 'equals', lakename.replace('-','.')).first(); 
        lakesize = lakerow.get('AREA_SKM').getInfo()
        summer = lakerow.get('time_perio').getInfo()
        # dumpclean(lakerow.getInfo()) #show metadata from this row

        print('Lake is %s km^2' % lakesize)
        # check which months are the summer months
        if summer == 'JFM':
            summer_months = [12,4]
        elif summer == 'JAS':
            summer_months = [6,10]
        feat= lakerow_shape.geometry();

        # get the overlapping summer LANDSAT image collection for lake polygon
        L5_filt = ee.ImageCollection('LANDSAT/LT5_L1T_TOA_FMASK').filterBounds(feat).filter(
            ee.Filter.calendarRange(summer_months[0],summer_months[1],'month'));

        # apply algorithms
        algName = ['ImpNIRwithSAC'];
        L5_filt_alg = L5_filt.map(calcAlgorithms);

        print ('Algorithms:')
        print(algName)
        print('...applied over Landsat 5 image collection.')

        # ----------------------------------------------------------------------------------------
        # Get 15-day composites
        # ----------------------------------------------------------------------------------------

        print('Generating 15-day composites...')

        # a) Create a bunch of features for the time periods we want means for, 
        #   each with a dateRange property.  

        #get list of dates for summer each year
        dateList = [];
        for yr in range(1984, 2012):
            comp_start = ee.Date.fromYMD(yr,summer_months[0],1);
            dateList.append(comp_start);
            comp_end = ee.Date;
            for i in range(1, 9):
                comp_end = comp_start.advance(16,'day')
                dateList.append(comp_end);
                comp_start = comp_end;

        dateRangeList = ee.List(dateList).map( 
            lambda (d): ee.DateRange(d,ee.Date(d).advance(16,'day'))
        );

        periods = dateRangeList.map(
            lambda (dr): ee.Feature(None,{'daterange': dr})
        );

        # print('a) list of features with daterange propery:',periods.getInfo())

        # b) SaveAll join those with the imagecollection (using dateRangeContains),

        # Create a time filter to define a match as within daterange
        periodFilter = ee.Filter.dateRangeContains(leftField='daterange',
                                                   rightField='system:time_start'
                                                  );

        # Define the join.
        saveAllJoin = ee.Join.saveAll(matchesKey='landsat',
                                      ordering='system:time_start',
                                      ascending=True,
                                      measureKey='period'
                                     );

        # Apply the join.
        landsatComp = saveAllJoin.apply(periods, L5_filt_alg, periodFilter);

        # print('b) Join.saveAll:', landsatComp.getInfo());

        # c) Map mean and set function over each of those, returning images.
        
        def getCompMean (feature1):
            compCol = ee.ImageCollection.fromImages(feature1.get('landsat'));
            compMean = compCol.mean(); #type: image

            #adjust how fmask_cloud and fmask_land are treated when creating composites
            compMean = compMean.select(ee.List(ee.Image(compMean).bandNames().remove('fmask_cloud')).remove('fmask_land'))
            compMean = compMean.addBands(compCol.select(['fmask_cloud']).min())
            compMean = compMean.addBands(compCol.select('fmask_land').max().where(compMean.select('fmask_water'),0))

            compCount = compCol.select(['fmask']).count().rename(['count']); #type: image 
            compMean = compMean.set(
                'comp_start',ee.DateRange(feature1.get('daterange')).start()).set(
                'comp_mid',ee.DateRange(feature1.get('daterange')).start().advance(8,'day')).set(
                'comp_end',ee.DateRange(feature1.get('daterange')).end()).set(
                'comp_num_images',compCol.size());
            return (compMean.addBands(compCount).addBands(ee.Image(1)));
        
        compMeans = ee.ImageCollection(landsatComp.map(getCompMean));
        
        # print('c) resultant imageCollection',compMeans.getInfo())

        # ----------------------------------------------------------------------------------------
        # Statistics from composites
        # ----------------------------------------------------------------------------------------

        # d) print dates from imageCollection to see which ranges were null

        compDates = compMeans.toList(1000).map(
            lambda (img): ee.Date(ee.Image(img).get('comp_mid')).format()
        );
        
        # print('d) composite dates without nulls: ')
        df = pd.DataFrame(compDates.getInfo(),columns=['date']);
        # print(df)

        compNums = compMeans.toList(1000).map(
            lambda (img): ee.Image(img).get('comp_num_images')
        );
        # print('e) number of Images in each period:', compNums.getInfo())
        
        if validation_regions:
             # Create historical image for validation
            print('Creating historical mean image...')
            compMeans_alg = compMeans.map(maskWater);
            compMeans_alg = compMeans_alg.select('ImpNIRwithSAC');
            histmean = compMeans_alg.mean()
            exp_img = ee.Image(histmean).clip(feat);
            exp_img_url = exp_img.getThumbURL(params={'min':0, 'max':0.01,'format':'png',
                                               'palette':'000000,0000ff,00ffff,00ff00,ffff00,ffa500,ff0000',
                                              }) 

            #visualize image in notebook
            display(Image(url=exp_img_url))

            #Import polygons for validation regions
            # Fusion Table with regions: "PolygonsForValidationRegions"
            # https://fusiontables.google.com/DataSource?docid=16V3xQruTtcugPsrSpRS8LuTglxxHbSw4fs834-XV
            regions_all = ee.FeatureCollection(
                "ft:16V3xQruTtcugPsrSpRS8LuTglxxHbSw4fs834-XV", "geometry")

            # Get region_polys from fusion table
            region = regions_all.filterMetadata('lakename', 'equals', lakename);
            region_polys = region.toList(6).map( lambda(poly): ee.Feature(poly).get('region') )
            print('List of regions:')
            dumpclean(region_polys.getInfo())

            # Get the mean of each region
            num_region_polys = region_polys.length().getInfo();
            print('Getting means of all {} regions'.format(num_region_polys))
            histmean_regions = histmean.reduceRegions(region,ee.Reducer.mean(),res)

            #start export process:
            MyTry=ee.batch.Export.table(histmean_regions, 
                                        lakename+'_region_means_python',
                                        {'fileFormat': 'CSV', 'driveFolder': 'GlobalBlooms'})
            MyTry.start()
            print('Export started:')
            dumpclean(MyTry.status())
            state = MyTry.status()['state']
            while state in ['READY', 'RUNNING']:
                print state + '...'
                time.sleep(60)
                state = MyTry.status()['state']
            print 'Done.'
            dumpclean(MyTry.status()) 
            print '------------------------------------------------------'
        
        elif exp_to_csv:

            # Get means + other statistics
            print('Getting csvs of means and other statistics...')

            # Map reducer over composite images
            stats = ee.FeatureCollection(compMeans.map(calcBloom));
        #         print(stats.getInfo());

            MyTry=ee.batch.Export.table(stats, lakename.replace('.','-')+'_15day_sum_ImpNIRwithSAC', 
                { 'fileFormat': 'CSV', 'driveFolder': 'GlobalBlooms'});
            MyTry.start()
            print('CSV export started:')
            dumpclean(MyTry.status())
            state = MyTry.status()['state']
            while state in ['READY', 'RUNNING']:
                print state + '...'
                time.sleep(60)
                state = MyTry.status()['state']
            print 'CSV export finished.'
            dumpclean(MyTry.status())
            toc = time.time();
            print 'Elapsed time: %s seconds' %(toc-tic)
    except:
        print "Unexpected error:", sys.exc_info()[0]
        raise #stops loop on error, e.g., HTTP connection error

print ('Done with all lakes')